In [ ]:
import numpy as np
import pennylane as qml
from sklearn.datasets import fetch_openml

from q_alchemy.pennylane_integration import QAlchemyStatePreparation, OptParams

mnist = fetch_openml('mnist_784', version=1, parser="auto")

zero: np.ndarray = mnist.data[mnist.target == "0"].iloc[0].to_numpy()
filler = np.empty(2 ** 10 - zero.shape[0])
filler.fill(0)

zero = np.hstack([zero, filler])
zero = zero / np.linalg.norm(zero)

dev = qml.device('lightning.qubit', wires=10)

@qml.qnode(dev)
def circuit(state=None):
    QAlchemyStatePreparation(
        state,
        wires=range(10),
        opt_params=OptParams(
            max_fidelity_loss=0.1,
            basis_gates=["id", "rx", "ry", "rz", "cx"],
            api_key="<your api key>"
        )
    )
    return qml.state()

print(qml.draw(circuit, level="device", max_length=100)(zero.tolist()))

0: ──────────────────────────────────────────────────────────────────────────────────────────────
1: ──────────────────────────────────────────────────────────────────────────────────────────────
2: ───────────────────────────────────────────────────╭X──RZ(-3.14)──RX(1.57)──RZ(3.14)──RX(1.57)
3: ──RZ(0.00)──RX(1.57)──RZ(3.17)──RX(1.57)──RZ(9.42)─╰●──RZ(0.00)───RX(1.57)──RZ(3.14)──RX(1.57)
4: ───────────────────────────────────────────────────╭X──RZ(3.14)───RX(1.57)──RZ(3.14)──RX(1.57)
5: ──RZ(0.00)──RX(1.57)──RZ(3.15)──RX(1.57)──RZ(9.42)─╰●──RZ(-3.14)──RX(1.57)──RZ(3.14)──RX(1.57)
6: ──RZ(0.00)──RX(1.57)──RZ(3.26)──RX(1.57)──RZ(9.42)─╭●──RZ(-3.14)──RX(1.57)──RZ(3.14)──RX(1.57)
7: ──RZ(0.00)──RX(1.57)──RZ(3.19)──RX(1.57)──RZ(9.42)─│──────────────────────────────────────────
8: ──RZ(0.00)──RX(1.57)──RZ(3.16)──RX(1.57)──RZ(9.42)─│──────────────────────────────────────────
9: ───────────────────────────────────────────────────╰X──RZ(0.00)───RX(1.57)──RZ(3.14)──RX(1.57)

───────────────────